In [ ]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, matplotlib
import matplotlib as mpl
import os
%matplotlib inline

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['pdf.fonttype']=42
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 11


In [ ]:
combined_df = pd.read_csv('.../calcium_imaging_data_heat.csv')
combined_df_AITC = pd.read_csv('.../calcium_imaging_data_AITC.csv')

In [ ]:
stim_df = pd.read_csv(p + "gcamp_heat_data_experiments.csv")

In [ ]:
example = 'Results_Copy of Copy of 20250723-3-ppk-Tdt-7F_00009tdTomato_Filteredsynchronized_video_gray.csv'

#plot example filename
example_group = combined_df[combined_df['filename'] == example]
fig = plt.figure(figsize=(4, 2))
plt.plot(example_group['frame'], example_group['dF/F_normalized'], label='dFF', color='red')
# Highlight stim periods using 'on' in combined_df.stim
stim_on_mask = example_group['stim'] == 'on'
if stim_on_mask.any():
    # Find contiguous 'on' periods
    on_frames = example_group.loc[stim_on_mask, 'frame']
    # Split into contiguous blocks
    blocks = np.split(on_frames, np.where(np.diff(on_frames) != 1)[0]+1)
    for block in blocks:
        if len(block) > 0:
            plt.axvspan(block.min(), block.max(), color='yellow', alpha=0.3, label='Stimulus Period')
# length of each block
            print(f'Block from {block.min()} to {block.max()} with length {len(block)}')


#plt.title(f'Example Filename: {example}')
plt.xlabel('Frame')
plt.ylabel('dFF')

plt.yticks([0.0, 0.5, 1.0, 1.5])

#plt.legend()
sns.despine(trim=True)
plt.show()

#fig.savefig(fig_path + 'example_20250723-3-ppk-Tdt-7F_00009_stim_periods.svg', dpi=600, transparent=True)

In [ ]:
example ='Results_Copy of Copy of 20250722-2-ppk-tdt-7F_00001tdTomato_Filteredsynchronized_video_gray.csv'


#plot example filename
example_group = combined_df[combined_df['filename'] == example]
fig = plt.figure(figsize=(4, 2))
plt.plot(example_group['frame'], example_group['dF/F_normalized'], label='dFF', color='black')
# Highlight stim periods using 'on' in combined_df.stim
stim_on_mask = example_group['stim'] == 'on'
if stim_on_mask.any():
    # Find contiguous 'on' periods
    on_frames = example_group.loc[stim_on_mask, 'frame']
    # Split into contiguous blocks
    blocks = np.split(on_frames, np.where(np.diff(on_frames) != 1)[0]+1)
    for block in blocks:
        if len(block) > 0:
            plt.axvspan(block.min(), block.max(), color='yellow', alpha=0.3, label='Stimulus Period')
            
            print(f'Block from {block.min()} to {block.max()} with length {len(block)}')


#plt.title(f'Example Filename: {example}')
plt.xlabel('Frame')
plt.ylabel('dFF')
#plt.legend()
plt.yticks([0.0, 0.5, 1.0, 1.5])
sns.despine(trim=True)
plt.show()

fig.savefig(fig_path + 'example_20250722-2-ppk-tdt-7F_00001_stim_periods.svg', dpi=600, transparent=True)

In [ ]:

# Parameters
pre_stim = 100
post_stim = 1500

aligned_traces = []

for _, row in stim_df.iterrows():
    stim_file = row['filename']
    stim_on = row['stim_on_frame']

    # Match based on clean filename
    match_df = combined_df[combined_df['filename'].str.contains(stim_file)]
    
    # Extract peri-stimulus window
    window_df = match_df[(match_df['frame'] >= stim_on - pre_stim) & 
                         (match_df['frame'] <= stim_on + post_stim)].copy()
    
    if len(window_df) == (pre_stim + post_stim + 1):
        window_df['aligned_frame'] = window_df['frame'] - stim_on
        window_df['trial_id'] = f"{stim_file}_{stim_on}"  # optional ID
        
        aligned_traces.append(window_df[['aligned_frame', 'dF/F_normalized', 'condition', 'trial_id']])

# Combine all windows
aligned_df = pd.concat(aligned_traces, ignore_index=True)

# Melt to long format for Seaborn
melted_df = pd.melt(
    aligned_df,
    id_vars=['aligned_frame', 'trial_id', 'condition'],
    value_vars=['dF/F_normalized'],
    var_name='Signal',
    value_name='dF/F'
)

# # Rename for clarity
# signal_labels = {
#     #'Ratio': 'Ratio (GCaMP/tdTomato)',
#     'dF/F_normalized': 'df/f Normalized'
# }
melted_df['Signal'] = melted_df['Signal']#.map(signal_labels)

# Plot
fig = plt.figure(figsize=(3, 2))
sns.lineplot(
    data=melted_df,
    x='aligned_frame',
    y='dF/F',
    hue='condition',
    palette=['black', 'red'],
    estimator='mean',
    ci=95,
    err_style='band'
)

plt.axvline(0, color='red', linestyle='--', label='Stim Onset')
plt.xlabel('Frame (Aligned to Stimulus Onset)')
plt.ylabel('ΔF/F')
#plt.title('Peri-Stimulus ΔF/F (GCaMP vs tdTomato)')
plt.legend(title='Signal',  bbox_to_anchor=(1.05, 1), loc='upper left')
sns.despine(trim=True)
plt.show()

fig.savefig(fig_path + 'dff_GCaMP_vs_tdTomato_all_time.svg', dpi=600, transparent=True)


In [ ]:
# Step 1: Filter to only stim ON periods
stim_on_df = combined_df[combined_df['stim'] == 'on'].copy()
fig = plt.figure(figsize=(1.75, 2.5))
sns.pointplot(
    data=stim_on_df,
    x='condition', 
    y='dF/F_normalized',
    hue='date_fly',
    palette='Greys',
    dodge=True,
    markeredgewidth=0,
    markers='o',
)
plt.legend(title='Fly', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylim([-0.1,.8])
sns.despine(trim=True)
plt.show()


from scipy import stats

# Filter to include only 'heat' and 'mechanical' conditions
filtered_df = stim_on_df[stim_on_df['condition'].isin(['heat', 'mechanical'])]

# Perform t-test on mechanical vs heat across all unique date_flies (paired by date_fly)
heat_means = []
mech_means = []
for date_fly in unique_date_flies:
    heat = filtered_df[(filtered_df['condition'] == 'heat') & (filtered_df['date_fly'] == date_fly)]['dF/F_normalized'].mean()
    mech = filtered_df[(filtered_df['condition'] == 'mechanical') & (filtered_df['date_fly'] == date_fly)]['dF/F_normalized'].mean()
    if not np.isnan(heat) and not np.isnan(mech):
        heat_means.append(heat)
        mech_means.append(mech)

# Paired t-test
t_stat, p_value = stats.ttest_rel(heat_means, mech_means)
print(f"Paired t-test across date_flies:")
print(f"Mean heat: {np.mean(heat_means):.3f}, Mean mechanical: {np.mean(mech_means):.3f}")
print(f"T-statistic: {t_stat:.3f}, p-value: {p_value:.4f}")


In [ ]:
# Set your folder path here
folder_path = 'H:/.shortcut-targets-by-id/10pxdlRXtzFB-abwDGi0jOGOFFNm3pmFK/Tuthill Lab Shared/Jessica/calcium imaging_Akira/analyzed_AITC_data'
output_list = []

# Choose how many initial frames to use for F0 baseline (commonly 10 or 30)
baseline_frames = 50

# Loop through all CSV files
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        df['filename'] = filename
        df['frame'] = df.index
        df['time_s'] = df['frame'] / 7.635  # synchronized video is also at 15.27 frames/second.
        split_cols = df['filename'].str.split('-', expand=True)

        # Assign columns as specified
        df['date'] = split_cols[0].str.extract('(\d+)')
        df['fly'] = split_cols[1]
        df['genotype'] = split_cols[2]
        # skip split_cols[3]
        df['trial'] = split_cols[5].str.split('_').str[1].str.extract('(\d+)')

        # Compute F0 for Mean1 (tdTomato) and Mean2 (GCaMP)
        F0_tdTomato = df['Mean1'].iloc[:baseline_frames].mean()
        F0_GCaMP = df['Mean2'].iloc[:baseline_frames].mean()

        # Compute ΔF/F for both channels
        df['dF/F_tdTomato'] = (df['Mean1'] - F0_tdTomato) / F0_tdTomato
        df['dF/F_GCaMP'] = (df['Mean2'] - F0_GCaMP) / F0_GCaMP

        # Compute ratio of GCaMP to tdTomato
        df['Ratio'] = df['Mean2'] / df['Mean1']

        # Compute F0 from first N frames of the ratio
        F0_ratio = df['Ratio'].iloc[:baseline_frames].mean()

        # Compute dF/F using the ratio
        df['dF/F_normalized'] = (df['Ratio'] - F0_ratio) / F0_ratio

        output_list.append(df[['frame','time_s','filename', 'date','fly', 'trial','genotype', 'Mean1', 'Mean2', 'Ratio','dF/F_tdTomato','dF/F_GCaMP', 'dF/F_normalized']])

combined_df_AITC = pd.concat(output_list, ignore_index=True)


combined_df_AITC['condition'] = 'AITC'
# fill "condition" with "mechanical" for files in mechanical_files
#combined_df.loc[combined_df['clean_filename'].isin(mechanical_files), 'condition'] = 'mechanical'


In [ ]:
from scipy.signal import savgol_filter

example = combined_df_AITC[combined_df_AITC['fly'] == '2']
group_sorted = example.sort_values(['trial', 'frame'])
fig = plt.figure(figsize=(3, 2))
# Only plot trials 1-10 for fly 2, apply Savitzky-Golay filter

filtered = group_sorted[group_sorted['trial'].between(1, 11)]
y = filtered['dF/F_normalized'].values
# window_length must be odd and less than or equal to the size of y
window_length = min(51, len(y) if len(y) % 2 == 1 else len(y) - 1)
if window_length < 3:
    window_length = 3  # minimum allowed
y_smooth = savgol_filter(y, window_length=51, polyorder=3)
plt.plot(filtered['frame'] + filtered['trial'] * 900, y_smooth, color='salmon')
# Optionally, add vertical lines to indicate trial boundaries
trial_ends = group_sorted.groupby('trial')['frame'].max() + group_sorted['trial'].unique() * 900
for t_end in trial_ends:
    plt.axvline(t_end, color='gray', linestyle='--', alpha=0.5)
#plt.title(f'Fly: {3}')
plt.xlabel('Frame (concatenated by trial)')
plt.ylabel('dF/F Normalized')
plt.xticks(np.arange(900, 12000, 900)),
plt.yticks([0.0, 1.0, 2])
sns.despine(trim=True)

plt.show()
fig.savefig(fig_path + 'example_fly_2_AITC_short.svg', dpi=600, transparent=True)

In [ ]:
#take the mean of each trial for each fly and plot as a pointplot of trial 1 vs the last trial per fly
mean_df = combined_df_AITC.groupby(['fly', 'trial']).agg({  
    'dF/F_normalized': 'max',
    'dF/F_tdTomato': 'mean',
    'dF/F_GCaMP': 'mean'
}).reset_index()


# Melt to long format for Seaborn
melted_df = pd.melt(
    mean_df,
    id_vars=['fly', 'trial'],
    value_vars=['dF/F_normalized', 'dF/F_tdTomato', 'dF/F_GCaMP'],
    var_name='Signal',
    value_name='Mean_dF/F'
)
# Map for pretty labels
signal_labels = {
    'dF/F_normalized': 'Normalized Ratio (GCaMP/tdTomato)',
    'dF/F_tdTomato': 'tdTomato',
    'dF/F_GCaMP': 'GCaMP'
}
melted_df['Signal'] = melted_df['Signal'].map(signal_labels)    
# Plot
fig = plt.figure(figsize=(2, 2))
sns.pointplot(
    data=melted_df[melted_df['Signal'] == 'Normalized Ratio (GCaMP/tdTomato)'],
    x='trial',
    y='Mean_dF/F',
    #hue='fly',
    palette=['salmon'],
    markers='o',
    linestyles='-',
    #dodge=True,
    #errorbar='se',
    #capsize=0.1,
    join=True
)
plt.xlabel('Trial')
plt.ylabel('Mean ΔF/F')
plt.title('Mean ΔF/F by Trial')
sns.despine(trim=True)
plt.show()

fig.savefig(fig_path + 'Mean_dFF_by_trial_AITC.svg', dpi=600, transparent=True)